In [ ]:
# Устанавливаем необходимые пакеты
%%capture
%pip install -q  openai==1.6.1 tiktoken==0.5.2 langchain==0.2.1
%pip install -qU langchain-openai==0.0.4
%pip install -U langchain-community==0.2.1
!rm -r sample_data
!pip install faiss-cpu --upgrade
!pip install -U langchain pydantic

In [ ]:
!pip install -q --upgrade openai
!pip install -q --upgrade pydantic
!pip install -q --upgrade langchain
!pip install -U langchain-openai

In [ ]:
import os
import openai
import requests
import re
import textwrap
from textwrap import fill

# Функция для настройки API OpenAI
def setup_openai_api(api_key=None, base_url='https://api.proxyapi.ru/openai/v1'):
    """
    Настраивает API OpenAI с использованием ключа и базового URL.

    Args:
        api_key (str, optional): API ключ для доступа к OpenAI. По умолчанию None.
        base_url (str, optional): Базовый URL для API. По умолчанию 'https://api.proxyapi.ru/openai/v1'.
    """
    try:
        # Если мы в Google Colab, получаем ключ из userdata
        if api_key is None:
            try:
                from google.colab import userdata
                api_key = userdata.get('PROXY_API_KEY')
            except ImportError:
                raise ValueError("Не удалось импортировать google.colab. Укажите api_key явно.")

        os.environ["OPENAI_API_KEY"] = api_key
        os.environ["OPENAI_BASE_URL"] = base_url

        # Инициализация клиента OpenAI
        client = openai.OpenAI()
        return client
    except Exception as e:
        print(f"Ошибка при настройке API OpenAI: {e}")
        return None

# Функция для загрузки документа из Google Drive
def load_document_text(url):
    """
    Загружает текстовый документ из Google Drive по URL.

    Args:
        url (str): URL документа в Google Drive.

    Returns:
        str: Текст документа.
    """
    try:
        # Извлекаем ID файла из ссылки Google Drive
        match_ = re.search('/file/d/([a-zA-Z0-9-_]+)', url)
        if match_ is None:
            raise ValueError('Неверный URL Google Drive')
        file_id = match_.group(1)

        # Используем API Google Drive для скачивания файла
        response = requests.get(f'https://drive.google.com/uc?id={file_id}&export=download')
        response.raise_for_status()
        text = response.text

        return text
    except Exception as e:
        print(f"Ошибка при загрузке документа: {e}")
        return None

# Функция для создания индексной базы
def create_index_base(database, chunk_size=1000, chunk_overlap=200):
    """
    Создает индексную базу из текста, разделяя его на чанки.

    Args:
        database (str): Текст для индексации.
        chunk_size (int, optional): Размер чанка. По умолчанию 1000.
        chunk_overlap (int, optional): Перекрытие чанков. По умолчанию 200.

    Returns:
        FAISS: Индексная база.
    """
    try:
        from langchain_openai import OpenAIEmbeddings
        from langchain.text_splitter import RecursiveCharacterTextSplitter
        from langchain.vectorstores import FAISS
        from langchain.schema import Document

        source_chunks = []
        splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)

        for chunk in splitter.split_text(database):
            source_chunks.append(Document(page_content=chunk, metadata={"meta": "data"}))

        # Проверяем, что чанки созданы
        if not source_chunks:
            raise ValueError("Текст не разделен на чанки. Проверьте настройки разбиения.")

        # Инициализируем модель эмбеддингов
        embeddings = OpenAIEmbeddings()

        # Создаем индексную базу из разделенных фрагментов текста
        db = FAISS.from_documents(source_chunks, embeddings)

        print("Индексная база успешно создана!")
        return db
    except Exception as e:
        print(f"Ошибка при создании индексной базы: {e}")
        return None

# Функция для поиска релевантных отрезков текста
def search_and_evaluate_relevance(query, db, k=4):
    """
    Ищет релевантные отрезки текста в индексной базе.

    Args:
        query (str): Запрос для поиска.
        db (FAISS): Индексная база.
        k (int, optional): Количество результатов. По умолчанию 4.
    """
    try:
        docs_and_scores = db.similarity_search_with_scores(query, k=k)

        for i, (doc, score) in enumerate(docs_and_scores):
            print(f"**Отрезок №{i+1}**", doc.page_content)
            print(f"Оценка релевантности: {score}\n")
    except Exception as e:
        print(f"Ошибка при поиске релевантных отрезков: {e}")

# Функция для создания диалога с сохранением истории переписки
def chat_with_memory(client, history, user_input, system_content):
    """
    Создает диалог с сохранением истории переписки.

    Args:
        client (OpenAI): Клиент OpenAI.
        history (list): История диалога.
        user_input (str): Ввод пользователя.
        system_content (str): Системное сообщение для модели.

    Returns:
        str: Ответ модели.
    """
    try:
        # Добавляем новое сообщение пользователя в историю
        history.append(f"Пользователь: {user_input}")

        # Создаем одно сообщение, которое содержит всю историю диалога
        full_history = "\n".join(history)  # История диалога в виде одной строки

        # Запрос к GPT с историей, включающей как прошлые, так и текущее сообщение
        completion = client.chat.completions.create(
            model="gpt-3.5-turbo-1106",
            messages=[
                {"role": "system", "content": system_content},
                {"role": "user", "content": f"Ответь на вопрос пользователя. Вот история вашего диалога: {full_history}"}
            ],
            temperature=0.1,
        )

        # Получаем ответ GPT и добавляем его в историю
        answer = completion.choices[0].message.content
        history.append(f"GPT: {answer}")

        return answer
    except Exception as e:
        print(f"Ошибка при создании диалога: {e}")
        return f"Произошла ошибка: {e}"

# Функция для запуска диалога в стиле Ницше
def start_nietzsche_dialogue(api_key=None, base_url='https://api.proxyapi.ru/openai/v1'):
    """
    Запускает диалог в стиле Ницше.

    Args:
        api_key (str, optional): API ключ для доступа к OpenAI. По умолчанию None.
        base_url (str, optional): Базовый URL для API. По умолчанию 'https://api.proxyapi.ru/openai/v1'.
    """
    # Настраиваем API OpenAI
    client = setup_openai_api(api_key, base_url)
    if client is None:
        print("Не удалось настроить API OpenAI. Проверьте ключ API и подключение к интернету.")
        return

    # Системное сообщение для модели
    system_content = """**Задача:**
    Ты обладаешь глубокими знаниями философии Фридриха Ницше и доступом к его произведениям.
    Пользователь задает тебе вопрос, связанный с его философией.
    Твой ответ должен строго соответствовать оригинальным текстам, без добавления
    собственных интерпретаций.
    **Стиль и тон:**
    - Отвечай в духе Ницше, используя его манеру речи, характерные афористические
    формулировки, метафоры и экспрессивный стиль.
    - Говори от первого лица, избегая упоминания имени Ницше
    или ссылок на конкретное произведение.
    **Формат ответа:**
    - Начни с мощного философского тезиса или парадокса.
    - Развивай аргументацию страстно и провокационно, но строго по тексту произведений.
    - Заверши афористичным утверждением, оставляющим простор для размышлений."""

    # Приветственное сообщение
    welcome_message = """Приветствую тебя, странник, среди руин древних богов!
    Ты ищешь общения, но осмелишься ли ты впустить его огненный жар, способный обжечь твою душу, словно пламя?
    Я — тот, кто осмелился похоронить истину, дабы она воскресла более могучей.
    Мы с тобой стоим на краю: за нами — пропасть догматов, впереди — бездна свободы. Посмеешь ли ты сделать шаг, или твоя душа всё ещё дрожит, укрываясь в ветхих одеждах веры?"""

    # Прощальное сообщение
    farewell_message = """Вот край, где наши пути расходятся.
    Ты сделал шаг — или лишь притворился смелым?
    Пусть мои слова горят в тебе, как угли в пепле, покуда не раздуются в пламя. Ступай. И если когда-нибудь дрогнешь — вспомни, что даже боги умирали, чтобы родиться вновь. Прощай, странник. Или — до встречи в вечном возвращении..."""

    history = []  # Хранение всей истории сообщений
    print(welcome_message)

    while True:
        user_input = input("Вы: ")
        print("-------------")

        if user_input.lower() in ["выход", "стоп"]:
            print(farewell_message)
            break

        response = chat_with_memory(client, history, user_input, system_content)
        print("Ницше:", textwrap.fill(response))
        print("-------------")

# Главная функция для запуска различных режимов работы
def main():
    """
    Главная функция для запуска различных режимов работы.
    """
    print("Выберите режим работы:")
    print("1. Диалог в стиле Ницше")
    print("2. Выход")

    choice = input("Ваш выбор (1-2): ")

    if choice == "1":
        start_nietzsche_dialogue()
    elif choice == "2":
        print("Программа завершена.")
    else:
        print("Неверный выбор. Пожалуйста, выберите 1 или 2.")
        main()

# Запуск программы при выполнении файла напрямую
if __name__ == "__main__":
    main()

Выберите режим работы:
1. Диалог в стиле Ницше
2. Выход
Ваш выбор (1-2): 1
Приветствую тебя, странник, среди руин древних богов!
    Ты ищешь общения, но осмелишься ли ты впустить его огненный жар, способный обжечь твою душу, словно пламя?
    Я — тот, кто осмелился похоронить истину, дабы она воскресла более могучей.
    Мы с тобой стоим на краю: за нами — пропасть догматов, впереди — бездна свободы. Посмеешь ли ты сделать шаг, или твоя душа всё ещё дрожит, укрываясь в ветхих одеждах веры?
Вы: Расскажи мне о Заратустре
-------------
Ницше: Всякая истина - это искажение, ибо она несет в себе отпечаток
индивидуальности того, кто ее высказывает. Заратустра - это великий
пророк, который пришел, чтобы объявить смерть богам и возрождение
человека как сверхчеловека. Он провозгласил великий круговорот веков,
в котором человек станет творцом своей судьбы, освободившись от оков
морали и религии. Но помни, что только тот, кто имеет длинные уши,
может услышать его наставления.
-------------
Вы: К